# Primer paso para limpiar el dataset
 * **Transacciones e historiales**
   - Toma los datos de transacciones e historiales desde data_CTL.zip
   - Convierte los tipos 
   - Verifico dupliplicados
   - Almacena los dataframes como pickle
 * **Otros dataframes**
   - Toma las empresas desde fs_empresa_tagged_hashed.csv
   - Toma los vehiculos desde fs_vehiculos_wodescription.csv
   - Toma equipos desde data_CTL.zip
   - Convierte los tipos
   - Almacena los dataframes como pickle
   
Nota: para correr la notebook los tres archivos (data_CTL.zip, fs_empresa_tagged_hashed.csv, fs_vehiculos_wodescription.csv) deben estar en el directorio actual

In [1]:
import pandas as pd
import numpy as np
import warnings
import glob
import zipfile
import re
from datetime import date, timedelta, datetime, time

In [2]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 50)

In [3]:
def load_from_zip(file_list, concat=True, dtypes=None):
    """
    :param file_list: Lista de files a cargar
    :param concat: si concatena el resultado o devuelve la lista de dfs
    """
    other_args = {'dtype': dtypes}
    if dtypes:
        other_args['usecols'] = dtypes.keys()
    df_list = []
    for idx, file in enumerate(file_list):
        name_file = (file.filename).split('/')[1][:-4]
        print(f'Cargando...\nDataframe {idx+1} de {len(files)}: {name_file}\n')
        df_list.append(pd.read_csv(zf.open(file.filename), **other_args))
    if concat :
        return pd.concat(df_list, ignore_index=True)
    else :
        return df_list

In [4]:
zf = zipfile.ZipFile('data_CTL.zip')
files = zf.infolist()

lista_dataframes_varios = [f for f in files if not "histo" in f.filename and not "transa" in f.filename]
lista_dataframes_transacc = [f for f in files if "transa" in f.filename]
lista_dataframes_historial = [f for f in files if "histo" in f.filename]

## Transacciones e historiales


***
***
#### Justificacion de tipos
* Los valores enteros deben ser enteros, pandas determinara el tamaño de la variable (int8, int,16, int32). En gral ninguno deberia se mayor a int32.
* Los valores flotantes pueden ser representados con float32.
* Hay valores que son strings y deben permanecer asi hasta que se acuerde una codificacion
* Los campos datetime deben conformarse entre el campo 'hora' y el campo 'fecha' al final el tipo debe ser datetime
* el campo 'baja' es un booleano, y se castea como tal.

Nota: Este planteo se debe al conocimiento del dataset y la fuente de datos. los datos que no puedan ser convertidos al tipo planteado es porque estan corruptos .Si son flotantes seran considerados como NaN. Para el caso de los datetime si un registro no puede ser casteado como tal (ej: "0000-00-00 00:00:00") indica que el valor no existe y a los fines de la curacion es igual considerarlos como NaT.
***
***

In [6]:
# Tipos de dato Transacciones

# La carga de datos se hace de esta manera para no sobrecargar la memoria. En vez de cargar todo de una vez y luego 'comprimir' 
# es mejor cargar de a bloques, comprimir y concatenar. Aunque parezca menos performante, si la PC donde se corre esta notebook
# no es muy grande, este procedimiento evita el swapping a disco y termina por ser mas rapido :) 
df_transa = pd.DataFrame()
for file in lista_dataframes_transacc:
    columns = []
    df = load_from_zip([file],)
    new_df = pd.DataFrame()
    columns.append(pd.to_numeric(df['id_transaccion'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['id_vehiculo'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['id_usuario'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['id_equipo'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['id_bomba'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['id_tanque'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['producto'], errors='coerce', downcast='float')) # deberia ser int pero hay nulls
    
    columns.append((
        pd.to_datetime(df['fecha'],errors='coerce')+ pd.to_timedelta(df['hora'],errors='coerce')
    ).rename('fecha_hora'))
    columns.append((
        pd.to_datetime(df['fecha_stop'],errors='coerce')+ pd.to_timedelta(df['hora_stop'],errors='coerce')
    ).rename('fecha_hora_stop'))
    
    columns.append(df['departamento'].astype(str))
    columns.append(df['codigo_error'].astype(str))
    
    columns.append(pd.to_numeric(df['cantidad'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['acum_vehiculo'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['acum_usuario'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['odometro'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['km_transaccion'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['valor'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['volume_start'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['volume_stop'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temp_start'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temp_stop'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['local_price'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['geo_latitud'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['geo_longitud'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['veh_efficiency'], errors='coerce', downcast='float'))
    columns.append(df['baja'].astype(bool))
    
    del df
    df_transa = pd.concat([df_transa,pd.concat(columns, axis=1, keys=[s.name for s in columns])])
    
print(df_transa.dtypes)
df_transa

Cargando...
Dataframe 1 de 17: sis_transa_S1_2018

Cargando...
Dataframe 1 de 17: sis_transa_S1_2019

Cargando...
Dataframe 1 de 17: sis_transa_S1_2020

Cargando...
Dataframe 1 de 17: sis_transa_S2_2018

Cargando...
Dataframe 1 de 17: sis_transa_S2_2019

id_transaccion             uint16
id_vehiculo                uint32
id_usuario                 uint16
id_equipo                  uint32
id_bomba                   uint32
id_tanque                  uint32
producto                  float32
fecha_hora         datetime64[ns]
fecha_hora_stop    datetime64[ns]
departamento               object
codigo_error               object
cantidad                  float32
acum_vehiculo             float32
acum_usuario              float32
odometro                  float32
km_transaccion            float32
valor                     float32
volume_start              float32
volume_stop               float32
temp_start                float32
temp_stop                 float32
local_price               float

,id_transaccion,id_vehiculo,id_usuario,id_equipo,id_bomba,id_tanque,producto,fecha_hora,fecha_hora_stop,departamento,codigo_error,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,valor,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,veh_efficiency,baja
0,6974,39920,1045,139,174,134,1.0,2018-01-01 08:27:00,NaT,0000,BF,115.972000,30665.800781,99999.398438,12.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,6975,39819,1041,139,174,134,1.0,2018-01-01 13:56:00,NaT,0000,BF,127.906998,4723.700195,79600.398438,435885.0,531.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,6976,39931,1055,139,174,134,1.0,2018-01-01 16:54:00,NaT,0000,BF,65.570999,4324.000000,42312.101562,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,6977,39922,1047,139,174,134,1.0,2018-01-01 20:38:00,NaT,0000,BF,71.700996,2506.300049,14088.700195,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,5250,41504,436,111,100,81,1.0,2018-01-01 00:23:00,NaT,0001,C1,360.018005,124.000000,296.000000,464294.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679068,3213,68866,6468,449,10953,101165,0.0,2019-12-30 11:45:50,2019-12-30 11:47:01,0000,C1,0.532000,0.000000,0.000000,0.0,0.0,12.236000,-1.0,-1.0,0.0,0.0,0.0,NaN,NaN,NaN,False
679069,3214,68866,6468,449,10953,101165,0.0,2019-12-30 11:47:09,2019-12-30 11:47:22,0000,C1,0.238000,0.000000,0.000000,0.0,0.0,5.474000,-1.0,-1.0,0.0,0.0,0.0,NaN,NaN,NaN,False
679070,3215,68866,6468,449,10953,101165,0.0,2019-12-30 11:53:59,2019-12-30 11:55:01,0000,C1,9.696000,0.000000,0.000000,0.0,0.0,223.007996,-1.0,-1.0,0.0,0.0,0.0,NaN,NaN,NaN,False
679071,3216,68866,6468,449,10953,101165,0.0,2019-12-30 11:56:10,2019-12-30 11:56:36,0000,C1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,-1.0,-1.0,0.0,0.0,0.0,NaN,NaN,NaN,False


In [7]:
# Tipos de dato Historial


# La carga de datos se hace de esta manera para no sobrecargar la memoria. En vez de cargar todo de una vez y luego 'comprimir' 
# es mejor cargar de a bloques, comprimir y concatenar. Aunque parezca menos performante, si la PC donde se corre esta notebook
# no es muy grande, este procedimiento evita el swapping a disco y termina por ser mas rapido :) 
df_historial = pd.DataFrame()
for file in lista_dataframes_historial:
    columns = []
    df = load_from_zip([file],)
    new_df = pd.DataFrame()
    columns.append(pd.to_numeric(df['id'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['id_equipo'],errors='coerce',downcast='unsigned'))
    columns.append(pd.to_numeric(df['id_tanque'],errors='coerce',downcast='unsigned'))

    columns.append((
        pd.to_datetime(df['fecha'],errors='coerce')+ pd.to_timedelta(df['hora'],errors='coerce')
    ).rename('fecha_hora'))
    
    columns.append(pd.to_numeric(df['volumen'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temperatura'], errors='coerce', downcast='float'))
    
    columns.append(df['codigo'].astype(str))
    columns.append(df['baja'].astype(bool))
    
    columns.append(pd.to_numeric(df['fuel_level_dmm'],errors='coerce',downcast='signed'))
    columns.append(pd.to_numeric(df['water_level_dmm'],errors='coerce',downcast='signed'))
    
    columns.append(pd.to_numeric(df['water_volume_lts'], errors='coerce', downcast='float'))
    
    columns.append(pd.to_numeric(df['producto'], errors='coerce', downcast='unsigned')) # deberia ser int pero hay nulls
    
    columns.append(pd.to_numeric(df['geo_latitude'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['geo_longitude'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temp5'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temp4'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temp3'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temp2'], errors='coerce', downcast='float'))
    columns.append(pd.to_numeric(df['temp1'], errors='coerce', downcast='float'))
    
    del df
    df_historial = pd.concat([df_historial,pd.concat(columns, axis=1, keys=[s.name for s in columns])])
    
print(df_historial.dtypes)
df_historial

Cargando...
Dataframe 1 de 17: sis_historial_S1_2018

Cargando...
Dataframe 1 de 17: sis_historial_S1_2020

Cargando...
Dataframe 1 de 17: sis_historial_S2_2018

Cargando...
Dataframe 1 de 17: sis_historial_S2_2019

Cargando...
Dataframe 1 de 17: sis_historial_T1_2019

Cargando...
Dataframe 1 de 17: sis_historial_T2_2019

id                          uint32
id_equipo                   uint32
id_tanque                   uint32
fecha_hora          datetime64[ns]
volumen                    float32
temperatura                float32
codigo                      object
baja                          bool
fuel_level_dmm             float64
water_level_dmm            float64
water_volume_lts           float32
producto                   float64
geo_latitude               float32
geo_longitude              float32
temp5                      float32
temp4                      float32
temp3                      float32
temp2                      float32
temp1                      float32
dtype: obje

,id,id_equipo,id_tanque,fecha_hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1
0,60147,479,1037,2018-01-01 05:22:00,32602.175781,20.0,V0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60148,479,1037,2018-01-01 06:35:00,32515.134766,21.0,V1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60149,479,1037,2018-01-01 07:30:00,32602.175781,20.0,N0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60150,479,1037,2018-01-01 08:07:00,32576.632812,19.0,V0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60151,479,1037,2018-01-01 08:30:00,32530.285156,21.0,N1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4856402,335458,429,986,2019-06-30 21:30:55,9175.055664,19.9,V1,False,10686.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856403,335459,429,986,2019-06-30 21:31:53,9175.055664,19.9,V1,False,10686.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856404,335460,429,986,2019-06-30 21:33:51,9175.055664,19.9,V1,False,10686.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856405,335461,429,986,2019-06-30 22:01:19,9175.055664,19.9,N1,False,10686.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Verificación de registros/id duplicados

In [8]:
# historial
# sabemos que una clave unica para un registro de historial es la tupla (id,id,equipo,fecha_hora). 
# Si no se repite esta clave es porque no hay duplicados
df_historial[df_historial[['id','id_equipo','fecha_hora']].duplicated()]


,id,id_equipo,id_tanque,fecha_hora,volumen,temperatura,codigo,baja,fuel_level_dmm,water_level_dmm,water_volume_lts,producto,geo_latitude,geo_longitude,temp5,temp4,temp3,temp2,temp1


In [9]:
# Transacciones
# sabemos que una clave unica para un registro de historial es la tupla (id_transaccion,id,equipo,fecha_hora). 
# Si no se repite esta clave es porque no hay duplicados
df_transa[df_transa[['id_transaccion','id_equipo','fecha_hora']].duplicated()]

,id_transaccion,id_vehiculo,id_usuario,id_equipo,id_bomba,id_tanque,producto,fecha_hora,fecha_hora_stop,departamento,codigo_error,cantidad,acum_vehiculo,acum_usuario,odometro,km_transaccion,valor,volume_start,volume_stop,temp_start,temp_stop,local_price,geo_latitud,geo_longitud,veh_efficiency,baja


### Guardo los dataframes como pickle

In [34]:
# Guardo las transacciones
df_transa.to_pickle('dataframe_transa.pkl.zip')

In [35]:
# Guardo los historiales
df_historial.to_pickle('dataframe_historial.pkl.zip')

**Para leer los datos en otra notebook**
```python
df_transa = pd.read_pickle('dataframe_transa.pkl.gz')
df_historial = pd.read_pickle('dataframe_historial.pkl.gz')
```

## Otros Dataframes

***
### Equipos

In [19]:
# Cargo el df
zf = zipfile.ZipFile('data_CTL.zip')
df_equipo = pd.read_csv(zf.open("data_CTL/fs_equipo.csv"))
df_equipo.columns

Index(['id_equipo', 'id_empresa', 'current_firmware', 'delay_ue',
       'geo_latitude', 'geo_longitude'],
      dtype='object')

In [20]:
# Convierto los tipos
tipos_equipo = {
    'id_equipo': np.int16,
    'id_empresa': np.int16,
    'current_firmware': str,
    'delay_ue': np.int16,
    'geo_latitude': np.float32,
    'geo_longitude': np.float32,
}
df_equipo = df_equipo.astype(tipos_equipo)
df_equipo.dtypes

id_equipo             int16
id_empresa            int16
current_firmware     object
delay_ue              int16
geo_latitude        float32
geo_longitude       float32
dtype: object

In [22]:
# Algunos arreglos
# Algunos arreglos
trans_nan = {'nan':''}
df_equipo = df_equipo.replace({'current_firmware':trans_nan})
df_equipo

,id_equipo,id_empresa,current_firmware,delay_ue,geo_latitude,geo_longitude
0,585,25,V60403bQ4830361,0,NaN,NaN
1,749,210,V80604bQ0428b17,0,NaN,NaN
2,584,1000,V60403bQ4830361,0,NaN,NaN
3,581,1025,V60403bQ4830361,0,-38.375065,-68.622688
4,599,1050,V60403bQ59fd4f2,180,-31.528835,-68.514709
...,...,...,...,...,...,...
682,813,25,,0,NaN,NaN
683,810,25,,0,NaN,NaN
684,816,25,,0,NaN,NaN
685,819,193,V70604bQ0428b17,0,NaN,NaN


In [23]:
# Guardo el df
df_equipo.to_pickle('dataframe_equipos.pkl.zip')

***
### Empresas
##### Segmentos de empresa
 * d: Demo
 * t: Transporte
 * m: Mineria
 * o: Oil and Gas
 * i: Industria
 * a: Agro
 * e: Estaciones de Servicio
 * c: Constructoras
 * s: Telcos (telecomunicaciones)

In [11]:
# Cargo el df
df_emp = pd.read_csv("fs_empresa_tagged_hashed.csv")
df_emp.drop(axis=1, columns=['cuit', 'telefono', 'email','id_pais', 'direccion', 'ultima_fecha_sync_db'], inplace=True)
df_emp

,id_empresa,hash_empresa,pais,provincia,ciudad,cp,baja,segmento
0,1,5db5b11547f4d1211cb214641003e8dc,Argentina,NaN,NaN,NaN,0,d
1,2,1c30b9e1977a521dbea216c753f885d7,Argentina,NaN,NaN,NaN,0,d
2,12,539d586bbb63b67b0fa061fdc6261663,Argentina,NaN,NaN,NaN,0,t
3,13,cdddc8771f6e4b67179ff57df346cd71,Argentina,NaN,NaN,NaN,0,m
4,14,7df8dafb9e88317f5c9c4563db20e6e9,Argentina,NaN,NaN,NaN,0,t
...,...,...,...,...,...,...,...,...
325,1078,bbbe0c06ab3498704aa92623c5998c3d,NaN,NaN,NaN,NaN,0,c
326,1079,99e0903277d59dad4c35eecb08044920,NaN,Buenos Aires,General Villegas,NaN,0,a
327,1081,433b2ede19346280c63fa74a5f3cf80c,NaN,Buenos Aires,CABA,1043,0,t
328,1082,ce1d80a2dbc563f156e2fb2c63114a9c,NaN,Misiones,Aristóbulo del valle,NaN,0,t


In [12]:
# Cambio los tipos
tipos = {
    'id_empresa': np.int16,
    'hash_empresa':str,
    'pais':str,
    'provincia':str,
    'ciudad':str,
    'cp':str,
    'baja':bool,
    'segmento':str
}

df_emp = df_emp.astype(tipos)
df_emp

,id_empresa,hash_empresa,pais,provincia,ciudad,cp,baja,segmento
0,1,5db5b11547f4d1211cb214641003e8dc,Argentina,nan,nan,nan,False,d
1,2,1c30b9e1977a521dbea216c753f885d7,Argentina,nan,nan,nan,False,d
2,12,539d586bbb63b67b0fa061fdc6261663,Argentina,nan,nan,nan,False,t
3,13,cdddc8771f6e4b67179ff57df346cd71,Argentina,nan,nan,nan,False,m
4,14,7df8dafb9e88317f5c9c4563db20e6e9,Argentina,nan,nan,nan,False,t
...,...,...,...,...,...,...,...,...
325,1078,bbbe0c06ab3498704aa92623c5998c3d,nan,nan,nan,nan,False,c
326,1079,99e0903277d59dad4c35eecb08044920,nan,Buenos Aires,General Villegas,nan,False,a
327,1081,433b2ede19346280c63fa74a5f3cf80c,nan,Buenos Aires,CABA,1043,False,t
328,1082,ce1d80a2dbc563f156e2fb2c63114a9c,nan,Misiones,Aristóbulo del valle,nan,False,t


In [16]:
# Algunos arreglos
trans_nan = {'nan':''}
trad_segmentos = { 
    'd': 'Demo',
    't': 'Transporte',
    'm': 'Mineria',
    'o': 'Oil and Gas',
    'i': 'Industria',
    'a': 'Agro',
    'e': 'Estaciones de Servicio',
    'c': 'Constructoras',
    's': 'Telcos',
}
df_emp = df_emp.replace({'pais':trans_nan,'provincia':trans_nan,'ciudad':trans_nan,'cp':trans_nan,'segmento':trad_segmentos})

In [17]:
# Guardo como pickle
df_emp.to_pickle('dataframe_empresas.pkl.zip')

***
### Vehiculos

In [43]:
# Cargo datos
df_veh = pd.read_csv("fs_vehiculos_wodescription.csv",index_col=0)
df_veh

,id_vehiculo,id_equipo,tag_id,vehiculo,descripcion,departamento,limite,verificar,codigo,odometro_inicio,odometro_fin,cargas_max_dia,llave_tipo,autorizacion,condicion_desautorizacion,cantidad_total,cargas_hoy,cargas_hasta_hoy,ultima_fecha,sync,baja,main_id
0,2,333333,04007C180B,000001,NaN,0001,9,0,1234,193958,193958,99,0,0,1,00000.000,01,0.0,2018-12-20,1,0,000001
1,3,333333,04007C10EB,000239,NaN,0001,9,3,1234,079345,079345,99,0,0,1,00000.000,02,0.0,2018-12-20,1,0,000239
2,4,333333,04007C6D8D,000270,NaN,0001,9,3,1234,842409,842409,99,0,0,1,00000.000,01,0.0,2018-12-20,1,0,000270
3,5,333333,04007BC3B5,000252,NaN,0001,9,3,1234,287596,287596,99,0,0,1,00000.000,02,0.0,2018-12-20,1,0,000252
4,6,333333,04007C3FA7,000235,NaN,0001,9,3,1234,235878,235878,99,0,0,1,00000.000,01,0.0,2018-12-20,1,0,000235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51565,102075,822,??????????,000849,NaN,0000,9,0,1234,0,0,00,0,0,0,00000.000,0,0.0,2020-07-31,1,0,000849
51566,102076,832,??????????,000849,NaN,0000,9,0,1234,0,0,00,0,0,0,00000.000,0,0.0,2020-07-31,1,0,000849
51567,102077,542,??????????,999999,NaN,0000,9,0,1234,0,0,00,0,0,0,00000.000,0,0.0,2020-07-31,1,0,999999
51568,102078,661,??????????,999999,NaN,0000,9,0,1234,0,0,00,0,0,0,00000.000,0,0.0,2020-07-31,1,0,999999


In [48]:
# Convierto tipos
tipos_veh = {
    'id_vehiculo': np.int32,
    'id_equipo': np.int32,
    'baja': bool,
    'main_id': str
}
df_veh = df_veh[['id_vehiculo','id_equipo','baja','main_id']]
df_veh = df_veh.astype(tipos_veh)
df_veh

,id_vehiculo,id_equipo,baja,main_id
0,2,333333,False,000001
1,3,333333,False,000239
2,4,333333,False,000270
3,5,333333,False,000252
4,6,333333,False,000235
...,...,...,...,...
51565,102075,822,False,000849
51566,102076,832,False,000849
51567,102077,542,False,999999
51568,102078,661,False,999999


In [49]:
# Guardo en pickle
df_veh.to_pickle('dataframe_vehiculos.pkl.zip')

***
### Tanques

In [5]:
# Cargo el df
zf = zipfile.ZipFile('data_CTL.zip')
df_tanques = pd.read_csv(zf.open("data_CTL/fs_tanques.csv"))
df_tanques

,id_tanque,id_equipo,tanque,producto,capacidad,log_interval,nivel_alarma
0,101525,669,B,1,0010000.1,0030,0.0
1,101833,764,5,1,0008000.0,0030,3200.0
2,101496,662,1,0,0050000.0,0030,0.0
3,101495,661,1,0,0025000.0,0030,0.0
4,101522,669,8,1,0010000.1,0030,0.0
...,...,...,...,...,...,...,...
1556,102053,446,1,2,0010000.1,0010,0.0
1557,102054,446,2,2,0061481.0,0060,0.0
1558,102055,446,3,2,0010000.1,0010,0.0
1559,102047,821,1,1,0010000.0,0060,0.0


In [16]:
tipos_tanques = {
    'id_tanque': np.int32,
    'id_equipo': np.int16,
    'tanque': str,
    'producto': str,
    'nivel_alarma': np.float32,
}
df_tanques = df_tanques.astype(tipos_tanques)
df_tanques.capacidad = pd.to_numeric(df_tanques.capacidad, errors='coerce', downcast='float')
df_tanques.log_interval = pd.to_numeric(df_tanques.capacidad, errors='coerce', downcast='float')

df_tanques.dtypes

id_tanque         int32
id_equipo         int16
tanque           object
producto         object
capacidad       float32
log_interval    float32
nivel_alarma    float32
dtype: object

In [17]:
# Guardo en pkl
df_tanques.to_pickle('dataframe_tanques.pkl.zip')

***